# 18 - Buck com MOSFET e bloco PWM

Exemplo objetivo de um buck com:

- chave **MOSFET** no estagio de potencia;
- fonte PWM eletrica para o gate;
- bloco virtual **`pwm_generator`** para gerar/comandar duty no backend.

A estrategia aqui e simples e robusta: a cada periodo de comutacao, o bloco PWM calcula o duty e atualizamos a fonte `Vgate` via `set_pwm_duty`.


In [1]:
from pathlib import Path
import sys

_root = Path.cwd()
_candidates = []
for _ in range(6):
    for _rel in (("build-test", "python"), ("build", "python")):
        candidate = _root / _rel[0] / _rel[1]
        if candidate.is_dir():
            _candidates.append(candidate)
    _root = _root.parent

_seen = set()
for candidate in _candidates:
    cstr = str(candidate)
    if cstr in _seen:
        continue
    _seen.add(cstr)
    if cstr not in sys.path:
        sys.path.insert(0, cstr)

import numpy as np
import matplotlib.pyplot as plt
import pulsim as ps

print(f"Pulsim version: {ps.__version__}")


Pulsim version: 0.3.3


In [ ]:
# Compat helper for Pulsim >=0.4 (Simulator API)
try:
    _PS = ps  # notebooks que usam alias 'ps'
except NameError:
    import pulsim as _PS

def run_transient_compat(circuit, t_start, t_stop, dt, x0=None, newton_options=None, linear_solver=None):
    """Bridge helper: mantem o retorno (times, states, success, msg) com a API atual."""

    def _legacy_call():
        args = []
        if x0 is not None:
            args.append(x0)
        if newton_options is not None:
            args.append(newton_options)
        if linear_solver is not None:
            args.append(linear_solver)
        return _PS.run_transient(circuit, t_start, t_stop, dt, *args)

    if hasattr(_PS, "SimulationOptions") and hasattr(_PS, "Simulator"):
        try:
            opts = _PS.SimulationOptions()
            opts.tstart = float(t_start)
            opts.tstop = float(t_stop)
            opts.dt = float(dt)

            if hasattr(opts, "dt_min"):
                opts.dt_min = min(float(opts.dt_min), float(dt))
            if hasattr(opts, "dt_max"):
                opts.dt_max = max(float(opts.dt_max), float(dt))

            if newton_options is not None:
                opts.newton_options = newton_options
            if linear_solver is not None and hasattr(opts, "linear_solver"):
                opts.linear_solver = linear_solver

            sim = _PS.Simulator(circuit, opts)
            if x0 is None:
                result = sim.run_transient()
            else:
                result = sim.run_transient(x0)
            if result.success or not hasattr(_PS, "run_transient"):
                return result.time, result.states, bool(result.success), str(result.message)

            # Fallback robusto para preservar notebooks mais antigos.
            return _legacy_call()
        except Exception:
            if hasattr(_PS, "run_transient"):
                return _legacy_call()
            raise

    return _legacy_call()


In [2]:
def create_buck_mosfet(Vin=24.0, fsw=100e3, duty=0.25, L=220e-6, C=220e-6, Rload=8.0):
    ckt = ps.Circuit()
    gnd = ckt.ground()

    n_in = ckt.add_node("vin")
    n_gate = ckt.add_node("gate")
    n_sw = ckt.add_node("sw")
    n_out = ckt.add_node("out")

    ckt.add_voltage_source("Vin", n_in, gnd, Vin)

    pwm = ps.PWMParams()
    pwm.v_high = 35.0
    pwm.v_low = 0.0
    pwm.frequency = fsw
    pwm.duty = duty
    ckt.add_pwm_voltage_source("Vgate", n_gate, gnd, pwm)

    m = ps.MOSFETParams()
    m.vth = 3.0
    m.kp = 0.35
    m.lambda_ = 0.01
    m.g_off = 1e-8
    ckt.add_mosfet("M1", n_gate, n_in, n_sw, m)

    ckt.add_diode("D1", gnd, n_sw, 350.0, 1e-9)
    ckt.add_inductor("L1", n_sw, n_out, L)
    ckt.add_capacitor("C1", n_out, gnd, C, 0.0)
    ckt.add_resistor("Rload", n_out, gnd, Rload)
    ckt.add_resistor("Rbleed_sw", n_sw, gnd, 1e6)

    return ckt, {"n_out": n_out, "n_sw": n_sw, "n_gate": n_gate}


In [3]:
Vin = 24.0
fsw = 100e3
Tsw = 1.0 / fsw
L = 220e-6
C = 220e-6
Rload = 8.0

duty_low = 0.26
duty_high = 0.48
step_period = 180
periods = 450

ckt, nodes = create_buck_mosfet(Vin=Vin, fsw=fsw, duty=duty_low, L=L, C=C, Rload=Rload)

# Bloco PWM virtual: recebe duty de comando e aplica clamp
ctrl = ps.Circuit()
n_cmd = ctrl.add_node("d_cmd")
ctrl.add_virtual_component(
    "pwm_generator",
    "PWM_MOD",
    [n_cmd],
    {
        "frequency": fsw,
        "duty_from_input": 1.0,
        "duty_gain": 1.0,
        "duty_offset": 0.0,
        "duty_min": 0.05,
        "duty_max": 0.90,
    },
    {},
)
x_ctrl = np.zeros(ctrl.system_size())

newton = ps.NewtonOptions()
newton.max_iterations = 140
dt = Tsw / 80.0

x = np.zeros(ckt.system_size())
t = 0.0

time_ms = []
vout_avg = []
vsw_avg = []
vgate_avg = []
duty_cmd_hist = []
duty_applied_hist = []
pwm_logic_hist = []

for k in range(periods):
    duty_cmd = duty_low if k < step_period else duty_high

    x_ctrl[n_cmd] = duty_cmd
    ctrl_step = ctrl.execute_mixed_domain_step(x_ctrl, t)
    duty_applied = float(ctrl_step.channel_values["PWM_MOD.duty"])
    pwm_logic = float(ctrl_step.channel_values["PWM_MOD"])

    ckt.set_pwm_duty("Vgate", duty_applied)

    times, states, ok, msg = run_transient_compat(ckt, t, t + Tsw, dt, x, newton)
    if not ok:
        rescue = ps.NewtonOptions()
        rescue.max_iterations = 320
        times, states, ok, msg = run_transient_compat(
            ckt, t, t + Tsw, dt * 0.5, x, rescue, robust=False, auto_regularize=True
        )

    if not ok:
        raise RuntimeError(f"Falha no periodo {k}: {msg}")

    states_arr = np.array(states)
    x = states_arr[-1].copy()
    t += Tsw

    time_ms.append(t * 1e3)
    vout_avg.append(float(np.mean(states_arr[:, nodes["n_out"]])))
    vsw_avg.append(float(np.mean(states_arr[:, nodes["n_sw"]])))
    vgate_avg.append(float(np.mean(states_arr[:, nodes["n_gate"]])))
    duty_cmd_hist.append(duty_cmd)
    duty_applied_hist.append(duty_applied)
    pwm_logic_hist.append(pwm_logic)

print("Simulacao concluida.")
print(f"Vout medio (ultimos 50 periodos): {np.mean(vout_avg[-50:]):.3f} V")
print(f"Duty aplicado final: {duty_applied_hist[-1]:.3f}")


Simulacao concluida.
Vout medio (ultimos 50 periodos): 11.915 V
Duty aplicado final: 0.480


In [4]:
time_ms = np.array(time_ms)
vout_avg = np.array(vout_avg)
duty_cmd_hist = np.array(duty_cmd_hist)
duty_applied_hist = np.array(duty_applied_hist)
pwm_logic_hist = np.array(pwm_logic_hist)

fig, ax = plt.subplots(3, 1, figsize=(11, 9), sharex=True)

ax[0].plot(time_ms, vout_avg, lw=2.0, color="#1f77b4", label="Vout medio/periodo")
ax[0].axvline(step_period * Tsw * 1e3, ls="--", color="#666", label="degrau de duty")
ax[0].set_ylabel("Vout [V]")
ax[0].grid(alpha=0.25)
ax[0].legend(loc="best")

ax[1].plot(time_ms, duty_cmd_hist, lw=1.8, color="#d62728", label="Duty comando")
ax[1].plot(time_ms, duty_applied_hist, lw=1.4, color="#2ca02c", label="Duty aplicado (PWM_MOD.duty)")
ax[1].set_ylabel("Duty")
ax[1].set_ylim(0.0, 1.0)
ax[1].grid(alpha=0.25)
ax[1].legend(loc="best")

ax[2].plot(time_ms, pwm_logic_hist, lw=1.2, color="#9467bd", label="PWM_MOD (0/1 no instante)")
ax[2].set_ylabel("PWM")
ax[2].set_xlabel("Tempo [ms]")
ax[2].set_ylim(-0.1, 1.1)
ax[2].grid(alpha=0.25)
ax[2].legend(loc="best")

plt.tight_layout()
plt.show()


/var/folders/0n/gs7hh4fj4bn8r3r4qsykw2840000gn/T/ipykernel_38828/3535002985.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
pre = slice(max(0, step_period - 60), step_period)
post = slice(max(0, len(vout_avg) - 60), len(vout_avg))

summary = {
    "Vout medio antes do degrau [V]": float(np.mean(vout_avg[pre])),
    "Vout medio depois do degrau [V]": float(np.mean(vout_avg[post])),
    "Duty medio antes": float(np.mean(duty_applied_hist[pre])),
    "Duty medio depois": float(np.mean(duty_applied_hist[post])),
}
summary


{'Vout medio antes do degrau [V]': 7.100940478380283,
 'Vout medio depois do degrau [V]': 11.91797596278682,
 'Duty medio antes': 0.26,
 'Duty medio depois': 0.48000000000000004}

## Observacoes

- Este exemplo mostra o **MOSFET no estagio de potencia** e o **bloco `pwm_generator` no backend**.
- O bloco PWM esta sendo usado para gerar/clamp de duty e a fonte eletrica `Vgate` recebe esse duty por `set_pwm_duty`.
- Para partir para malha fechada, basta trocar `duty_cmd` por um valor vindo de `pi_controller` (ou `pid_controller`).
